In [ ]:
import pandas as pd
import numpy as np

# 读数据

In [ ]:
# 读取数据
open_pathA = r'→TRJ_readerA.csv'
open_pathB = r'→TRJ_readerB.csv'

df_A = pd.read_csv(open_pathA,index_col=0)
df_B = pd.read_csv(open_pathB,index_col=0)

print(f'删除前俩表维度情况'+str(df_A.shape)+str(df_B.shape))

In [ ]:
df_A

# 选出重复特征 保存成txt

In [ ]:
duplicate_history = []

df_tmp = df_A.copy(deep=True)

In [ ]:
# 单个变量重复
print(f'{df_tmp.shape[1]}条数据中，离散变量某类别数量')
threshold_duplicate = 0.75
duplicate_col = []  # 存储需要删除的列

duplicate_condition_col = []
for col in df_tmp.columns:
    max_num = df_tmp[col].value_counts()  # 单个类别每一类的数量
    num_ratio = max_num.max() / max_num.sum()  # 最多类别占总数的比例
    duplicate_condition_col.append(num_ratio)
    if num_ratio > threshold_duplicate:
        duplicate_col.append(col)
duplicate_condition = pd.DataFrame(duplicate_condition_col).set_index(df_tmp.columns)
duplicate_condition.columns = ['单特征最高占比']

print(f'   待删除的列{duplicate_col}有{len(duplicate_col)}个')

duplicate_condition = duplicate_condition.sort_values(by="单特征最高占比",ascending=False)
duplicate_condition.iloc[1:20,:]

In [ ]:
# # # 手动增加,删除一些
# duplicate_col.append('lbp-3D-m2_firstorder_Maximum')
# duplicate_col.append('lbp-3D-m2_firstorder_Range')

# 删除指定列
df_tmp.drop(columns=duplicate_col, inplace=True)      # inplace 要不要在原数据上操作
duplicate_history.append(duplicate_col)  # 将删除记录加进历史
print("已删除满足条件的列，并且放进历史中:", duplicate_col)
duplicate_col=[]


In [ ]:
len(duplicate_history)

In [ ]:
# 保存上述删掉的特征
duplicate_history_set = []
for i in duplicate_history:
    for j in i:
        duplicate_history_set.append(j)
duplicate_history_set = list(set(duplicate_history_set))

In [ ]:
df_duplicate_history = pd.DataFrame(data={'Feature':duplicate_history_set})
df_duplicate_history.to_csv('→特征重复删除.txt',sep=' ', index=False,header=False)
print(f'共删掉了{len(duplicate_history_set)}个特征，已保存到【→特征重复删除.txt】')

df_duplicate_history

# 保存

In [ ]:
list_duplicate = pd.read_table('→特征重复删除.txt',index_col=None,header=None)[0].tolist()
print(len(list_duplicate))
list_duplicate

In [ ]:
def saveRet(df_demo,list_duplicate,save_name):
    print(f'原始维度的特征为{df_demo.shape}，要删除{len(list_duplicate)}个，期待保留{df_demo.shape[1]-len(list_duplicate)}个')
    list_selectedFeature = list(set(df_demo.columns.tolist()) - set(list_duplicate))
    
    df_demo = df_demo[list_selectedFeature]
    print(f'筛选后{df_demo.shape}')
    
    df_demo.to_csv(save_name, index=True)
    print(f'已经保存到{save_name}')

In [ ]:
saveRet(df_A,list_duplicate,'→'+open_pathA[:-4]+'_dup'+str(threshold_duplicate)+'.csv')

In [ ]:
saveRet(df_B,list_duplicate,'→'+open_pathB[:-4]+'_dup'+str(threshold_duplicate)+'.csv')